In [ ]:
import pandas as pd
import zipfile
import sqlite3
import os
# non IMDB 
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
tmbd = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tn_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
rt_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep='\t')
rt_review = pd.read_csv('zippedData/rt.reviews.tsv.gz',sep='\t',encoding='latin-1')

In [ ]:
extraction_path = 'zippedData/' 
zip_file_path = 'zippedData/im.db.zip'
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# The .db file should now be in the extraction_path directory
# Assuming the .db file has the same name as the .zip file
db_file_name = os.path.basename(zip_file_path).replace('.zip', '')
db_file_path = os.path.join(extraction_path, db_file_name)

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Perform database operations
cursor = conn.cursor()

# For example, to print the list of tables
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

query_movie_basics = ("SELECT * FROM movie_basics")
query_directors = ("SELECT * FROM directors")
query_known_for = ("SELECT * FROM known_for")
query_movie_akas = ("SELECT * FROM movie_akas")
query_movie_ratings = ("SELECT * FROM movie_ratings")
query_persons = ("SELECT * FROM persons")
query_principals = ("SELECT * FROM principals")
query_writers = ("SELECT * FROM writers")

big_query = (
"""
SELECT 
movie.movie_id,
movie.start_year,
movie.primary_title,
movie.original_title,
movie.genres,
movie.runtime_minutes,
MAX(IIF(principals.category='director',persons.primary_name,NULL)) as director_name,
MAX(IIF(principals.category='writer',persons.primary_name,NULL)) as writer_name,
GROUP_CONCAT(IIF(principals.category='actor',persons.primary_name,NULL),', ') as actors
FROM movie_basics as movie
LEFT JOIN principals 
    ON movie.movie_id = principals.movie_id 
LEFT JOIN persons 
    ON persons.person_id = principals.person_id
GROUP BY 1
"""
)



movie_basics_df = pd.read_sql_query(query_movie_basics,conn)
directors_df = pd.read_sql_query(query_directors,conn)
known_for_df = pd.read_sql_query(query_known_for,conn)
movie_akas_df = pd.read_sql_query(query_movie_akas,conn)
movie_ratings_df = pd.read_sql_query(query_movie_ratings,conn)
persons_df = pd.read_sql_query(query_persons,conn)
principals_df = pd.read_sql_query(query_principals,conn)
writers_df = pd.read_sql_query(query_writers,conn)

big_df = pd.read_sql_query(big_query,conn)

conn.close()

In [ ]:
big_df.head()

In [ ]:
movie_basics_df.head()

In [ ]:
bom.info()

In [ ]:
tmbd.info()

In [ ]:
tn_budgets.info()

In [ ]:
rt_info.info()

In [ ]:
rt_review.info()

In [ ]:
# need to check for movie name title discrepencies 

In [ ]:
rt_review.head()

In [ ]:
rt_info.head()

In [ ]:
#could be useful for foreign language films
movie_akas_df.head()

In [ ]:
directors_df.head()

In [ ]:
known_for_df.head()

In [ ]:
persons_df.head()

In [ ]:
movie_basics_df.head()

In [ ]:
tn_budgets.head()

In [ ]:
#Use tn_budgets for production cost, domestic gross, foreign gross, release date
#directors_df combined with principles_df for directors
#movie_basics_df has runtime and genre, and title
#persons_df has profession and name (for actors)
#movie_ratings_df has user ratings
#connecting tn_budgets to imdb databases with tn_budgets movie to movie basics origional title or primary title
#need to establish budget floor and release date floor
#not using rotten tomatoes as it has no movie title for some reason

In [ ]:
tn_budgets.head()

tn_budgets['year'] = pd.DatetimeIndex(tn_budgets['release_date']).year

tn_budgets[tn_budgets['movie']=='On the Road']

In [ ]:
merged_df = pd.merge(big_df, tn_budgets, left_on=['primary_title','start_year'],right_on=['movie','year'])


gross = pd.to_numeric(merged_df['worldwide_gross'].str.replace('\$', '', regex=True).str.replace(',',''))
production = pd.to_numeric(merged_df['production_budget'].str.replace('\$', '', regex=True).str.replace(',',''))
domestic = pd.to_numeric(merged_df['domestic_gross'].str.replace('\$', '', regex=True).str.replace(',',''))

merged_df['worldwide_gross'] = gross
merged_df['production_budget'] = production
merged_df['domestic_gross'] = domestic
merged_df['Profit'] = gross - production


merged_df['Profit:Production Ratio'] = (merged_df['Profit']/production)

merged_df = merged_df[merged_df['worldwide_gross']!='$0']

merged_df['release_date'] = pd.to_datetime(merged_df['release_date'])

merged_df = merged_df.dropna(subset=['genres'])
merged_df = merged_df.dropna(subset=['runtime_minutes'])

merged_df = merged_df[merged_df['release_date'] >= '2000-01-01']

merged_df = merged_df[merged_df['production_budget'] >= 1000000]

merged_df